In [1]:
# Import libraries
import os
import requests
import dlt
import lancedb
from dlt.destinations.adapters import lancedb_adapter
from rest_api import RESTAPIConfig, rest_api_source
from dlt.sources.helpers.rest_client.paginators import BasePaginator, JSONResponsePaginator
from dlt.sources.helpers.requests import Response, Request
from datetime import datetime, timezone
import ollama

In [2]:
!yes | dlt init rest_api lancedb

Looking up the init scripts in https://github.com/dlt-hub/verified-sources.git...
No files to update, exiting
yes: standard output: Broken pipe


In [3]:
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"
os.environ["DESTINATION__LANCEDB__CREDENTIALS__URI"] = ".lancedb"

In [4]:
# Ensure the API key is set in the environment variables
api_key = os.getenv("SOURCES__REST_API__NOTION__API_KEY")
if not api_key:
    raise EnvironmentError("The Notion API key is not set in the environment variables.")

In [5]:
class PostBodyPaginator(BasePaginator):
    def __init__(self):
        super().__init__()
        self.cursor = None

    def update_state(self, response: Response) -> None:
        # Assuming the API returns an empty list when no more data is available
        if not response.json():
            self._has_next_page = False
        else:
            self.cursor = response.json().get("next_cursor")
            if self.cursor is None:
                self._has_next_page = False

    def update_request(self, request: Request) -> None:
        if request.json is None:
            request.json = {}

        # Add the cursor to the request body
        request.json["start_cursor"] = self.cursor

        
# API Configuration
@dlt.resource(name="employee_handbook")
def rest_api_notion_resource():
    notion_config: RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": api_key
            },
            "headers":{
            "Content-Type": "application/json",
            "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "workshop",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    },
                }
            }
        ]
    }

    yield from rest_api_source(notion_config,name="employee_handbook")
    

# Function that describes how to extract the data and the structure of the output
def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    if block_type != "paragraph":
        content = ""
    else:
        try:
            content = response["paragraph"]["rich_text"][0]["plain_text"]
        except IndexError:
            content = ""
    return {
        "block_id": block_id,
        "block_type": block_type,
        "content": content,
        "last_edited_time": last_edited_time,
        "inserted_at_time": datetime.now(timezone.utc)
    }


@dlt.resource(
    name="employee_handbook",
    write_disposition="merge", # appends data
    primary_key="block_id",
    columns={"last_edited_time":{"dedup_sort":"desc"}} # in case of duplicate data, takes the latest version
    )

# Defines the incremental loading
def rest_api_notion_incremental(
    last_edited_time = dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z",primary_key=("block_id"))
):
    # last_value = last_edited_time.last_value
    # print(last_value)

    for block in rest_api_notion_resource.add_map(extract_page_content):
        if not(len(block["content"])):
            continue
        yield block

        
# Pipeline that extracts and loads data to lancedb       
def load_notion() -> None:
    pipeline = dlt.pipeline(
        pipeline_name="company_policies",
        destination="lancedb",
        dataset_name="notion_pages",
        # full_refresh=True
    )

    load_info = pipeline.run(
        lancedb_adapter(
            rest_api_notion_incremental,
            embed="content"
        ),
        table_name="employee_handbook",
        write_disposition="merge"
    )
    print(load_info)

load_notion()

Pipeline company_policies load step completed in ---
0 load package(s) were loaded to destination LanceDB and into dataset None
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x7f0fc19a8910> location to store data


In [6]:
db = lancedb.connect(".lancedb")
print(db.table_names())

['notion_pages____dlt_loads', 'notion_pages____dlt_pipeline_state', 'notion_pages____dlt_version', 'notion_pages___dltSentinelTable', 'notion_pages___employee_handbook']


In [7]:
dbtable = db.open_table("notion_pages___employee_handbook")
dbtable.to_pandas()

,id__,vector__,block_id,block_type,content,last_edited_time,inserted_at_time,_dlt_load_id,_dlt_id
0,71e89a85-ae0b-5b68-866b-bd3922ec7548,"[-0.058588073, -0.075404495, 0.03377525, 0.009...",c0262981-b5f1-4a57-a91f-2e75f649b86c,paragraph,Our company operates between 9 a.m. to 7 p.m. ...,2024-07-18 14:00:00+00:00,2024-07-21 16:09:12.876331+00:00,1721578152.1282816,ORTkWQcvXdRplQ
1,a28e913f-761f-5684-8cd5-0d0c49e0338c,"[-0.0049689133, -0.0039119944, 0.028705683, 0....",faacf4ec-90be-4e96-b8b9-29b5112bc7ca,paragraph,Employees receive [20 days] of Paid Time Off (...,2024-06-26 09:03:00+00:00,2024-07-21 16:09:12.880214+00:00,1721578152.1282816,v6fsegn1tQEqQQ
2,a18932d9-1583-5c42-bd0d-0f96738c5e6c,"[0.03206092, 0.024244629, 0.00847134, 0.031790...",e6021a51-f403-4950-80c2-ebff005c7289,paragraph,Our company observes the following holidays: N...,2024-06-26 09:08:00+00:00,2024-07-21 16:09:12.880389+00:00,1721578152.1282816,nHUNLsUsZu4MIw
3,93661874-13a2-5a43-bed8-868005dfd5e2,"[-0.013155231, 0.008382412, 0.017044384, 0.051...",b8f4cc6d-c28c-4071-9545-caadce5eb37b,paragraph,These holidays are considered “off-days” for m...,2024-06-26 09:09:00+00:00,2024-07-21 16:09:12.880522+00:00,1721578152.1282816,SRSqhwyWwRvdkg
4,b220778f-1118-5c22-b614-3bc0fd0a602b,"[0.027987536, 0.06734364, 0.03980643, 0.007744...",ea7a1beb-6874-4f41-966d-dc1f80a1f635,paragraph,Employees who are unable to work due to illnes...,2024-06-26 09:11:00+00:00,2024-07-21 16:09:12.880672+00:00,1721578152.1282816,nKYOvLC4D9f8kQ
5,d0f801ba-d3cc-5252-ad6e-3285662b609c,"[0.032526128, 0.008159533, 0.08443569, 0.05564...",bd7a9110-fac5-4270-9493-4039ca67b467,paragraph,Losing a loved one is traumatizing. If this ha...,2024-06-26 09:17:00+00:00,2024-07-21 16:09:12.880801+00:00,1721578152.1282816,FAPud4qcY6UDMg
6,579b97f9-a5e2-53af-b4f7-efc9ad5105ad,"[-0.0073140473, 0.014710639, -0.019091226, 0.0...",b1718dee-8c0f-4189-8c75-0e8c7844a501,paragraph,"In accordance with German law, we offer a comp...",2024-06-26 09:20:00+00:00,2024-07-21 16:09:12.880926+00:00,1721578152.1282816,BQR16WvhFX/z9w
7,a9083b7e-22cc-5b1f-8040-cb7aa1f72338,"[-0.03153839, 0.034259915, -0.027282672, 0.027...",5bfa90c5-461d-406a-9324-a1dd54bad0d5,paragraph,We recognize the vital role that fathers and p...,2024-06-26 09:21:00+00:00,2024-07-21 16:09:12.881051+00:00,1721578152.1282816,HVRCm9YG4LKTTg
8,6adeb540-d180-5d40-bc84-c40e5c173ea1,"[-0.038923874, 0.1208174, 0.04620858, -0.00543...",baac0ba4-9b60-450e-8cc1-1e6e2a0fb7d9,paragraph,"In this section, we describe what we offer to ...",2024-07-03 17:34:00+00:00,2024-07-21 16:09:13.077992+00:00,1721578152.1282816,sYbiHUJJoPIQlA
9,cffdb1bb-a146-5e90-8fbb-a1d577a2a98e,"[-0.07571496, 0.14543605, 0.0011521688, -0.024...",0e429073-6383-4918-8961-fcc66346067f,paragraph,{edited} Employee health is important to us. W...,2024-07-18 17:28:00+00:00,2024-07-21 16:09:13.078253+00:00,1721578152.1282816,iT8FRSJVoYwmtA


In [8]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
######################################################################## 100.0%#=#=#                                                                         ##################################################         91.2%
>>> Installing ollama to /usr/local/bin...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [9]:
%%capture
!ollama pull llama2-uncensored

In [10]:
def retrieve_context_from_lancedb(dbtable, question, top_k=2):

    query_results = dbtable.search(query=question).to_list()
    context = "\n".join([result["content"] for result in query_results[:top_k]])

    return context

In [11]:
def main():
  # Connect to the lancedb table
  db = lancedb.connect(".lancedb")
  dbtable = db.open_table("notion_pages___employee_handbook")

  # A system prompt telling ollama to accept input in the form of "Question: ... ; Context: ..."
  messages = [
      {"role": "system", "content": "You are a helpful assistant that helps users understand policies inside a company's employee handbook. The user will first ask you a question and then provide you relevant paragraphs from the handbook as context. Please answer the question based on the provided context. For any details missing in the paragraph, encourage the employee to contact the HR for that information. Please keep the responses conversational."}
  ]

  while True:
    # Accept user question
    question = input("You: ")

    # Retrieve the relevant paragraphs on the question
    context = retrieve_context_from_lancedb(dbtable,question,top_k=2)

    # Create a user prompt using the question and retrieved context
    messages.append(
        {"role": "user", "content": f"Question: '{question}'; Context:'{context}'"}
    )

    # Get the response from the LLM
    response = ollama.chat(
        model="llama2-uncensored",
        messages=messages
    )
    response_content = response['message']['content']
    print(f"Assistant: {response_content}")

    # Add the response into the context window
    messages.append(
        {"role": "assistant", "content":response_content}
    )

In [ ]:
main()